
<a href="https://colab.research.google.com/drive/10c1amlVfvev7ovrD3iOXjv7UNZXVEA8o?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [1]:
#1 Install necessary libraries
!pip install -q unsloth "torch>=2.1" transformers peft datasets trl accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Step 2: Import necessary libraries

from unsloth import FastLanguageModel
from datasets import Dataset, load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
import torch
import matplotlib.pyplot as plt


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Step 3A: Load TinyLlama Model

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.4.1 patched 22 layers with 22 QKV layers, 22 O layers and 0 MLP layers.


In [14]:
# Step 3B: General knowledge fine-tuning data

general_knowledge = [
    {"text": "User: What is the boiling point of water?\nAssistant: Water boils at 100 degrees Celsius at standard pressure."},
    {"text": "User: Who discovered gravity?\nAssistant: Sir Isaac Newton is credited with the discovery of gravity."},
    {"text": "User: What is the capital of France?\nAssistant: Paris is the capital city of France."},
    {"text": "User: How many continents are there?\nAssistant: There are seven continents on Earth."},
    {"text": "User: What gas do plants use for photosynthesis?\nAssistant: Plants use carbon dioxide for photosynthesis."},
    {"text": "User: What is the square root of 64?\nAssistant: The square root of 64 is 8."},
    {"text": "User: Who painted the Mona Lisa?\nAssistant: Leonardo da Vinci painted the Mona Lisa."},
    {"text": "User: What is the fastest land animal?\nAssistant: The cheetah is the fastest land animal."},
    {"text": "User: What is the currency of Japan?\nAssistant: The currency of Japan is the Yen."},
    {"text": "User: What organ pumps blood through the body?\nAssistant: The heart pumps blood through the body."},
    {"text": "User: Who was the first person to walk on the Moon?\nAssistant: Neil Armstrong was the first person to walk on the Moon."},
    {"text": "User: How many sides does a hexagon have?\nAssistant: A hexagon has six sides."},
    {"text": "User: What is the process of water turning into vapor called?\nAssistant: It is called evaporation."},
    {"text": "User: What metal is liquid at room temperature?\nAssistant: Mercury is liquid at room temperature."},
    {"text": "User: What is the main ingredient in guacamole?\nAssistant: Avocado is the main ingredient in guacamole."},
    {"text": "User: How many legs does a spider have?\nAssistant: A spider has eight legs."},
    {"text": "User: What is the largest mammal?\nAssistant: The blue whale is the largest mammal."},
    {"text": "User: What gas do humans need to breathe?\nAssistant: Humans need oxygen to breathe."},
    {"text": "User: How many days are in a leap year?\nAssistant: A leap year has 366 days."},
    {"text": "User: What is the freezing point of water?\nAssistant: Water freezes at 0 degrees Celsius."},
    {"text": "User: What color is chlorophyll?\nAssistant: Chlorophyll is green."},
    {"text": "User: Which planet is known as the Red Planet?\nAssistant: Mars is known as the Red Planet."},
    {"text": "User: What is the chemical symbol for gold?\nAssistant: The chemical symbol for gold is Au."},
    {"text": "User: How many bones are there in the adult human body?\nAssistant: There are 206 bones in the adult human body."},
    {"text": "User: Who wrote the play 'Romeo and Juliet'?\nAssistant: William Shakespeare wrote 'Romeo and Juliet'."},
    {"text": "User: What is the largest ocean on Earth?\nAssistant: The Pacific Ocean is the largest ocean on Earth."},
    {"text": "User: What type of animal is a Komodo dragon?\nAssistant: A Komodo dragon is a species of lizard."},
    {"text": "User: What is the capital city of Canada?\nAssistant: Ottawa is the capital city of Canada."},
    {"text": "User: What is H2O commonly known as?\nAssistant: H2O is commonly known as water."},
    {"text": "User: How many hours are there in a day?\nAssistant: There are 24 hours in a day."},
    {"text": "User: What is the smallest planet in our solar system?\nAssistant: Mercury is the smallest planet in our solar system."},
    {"text": "User: Who invented the telephone?\nAssistant: Alexander Graham Bell invented the telephone."},
    {"text": "User: What shape has three sides?\nAssistant: A triangle has three sides."},
    {"text": "User: What is the chemical formula for table salt?\nAssistant: The chemical formula for table salt is NaCl."},
    {"text": "User: What is the tallest mountain in the world?\nAssistant: Mount Everest is the tallest mountain in the world."},
    {"text": "User: What is the main language spoken in Brazil?\nAssistant: Portuguese is the main language spoken in Brazil."},
    {"text": "User: What is the process by which plants make food?\nAssistant: Photosynthesis is the process by which plants make food."},
    {"text": "User: What is the hottest planet in our solar system?\nAssistant: Venus is the hottest planet in our solar system."},
    {"text": "User: What is the name of the fairy in Peter Pan?\nAssistant: Tinker Bell is the fairy in Peter Pan."},
    {"text": "User: Who wrote 'The Origin of Species'?\nAssistant: Charles Darwin wrote 'The Origin of Species'."},
    {"text": "User: What is the process of a caterpillar becoming a butterfly called?\nAssistant: It is called metamorphosis."},
    {"text": "User: Which element has the chemical symbol O?\nAssistant: Oxygen has the chemical symbol O."},
    {"text": "User: What is the main ingredient in bread?\nAssistant: Flour is the main ingredient in bread."},
    {"text": "User: How many rings are there on the Olympic flag?\nAssistant: There are five rings on the Olympic flag."},
    {"text": "User: What device measures temperature?\nAssistant: A thermometer measures temperature."},
    {"text": "User: What is the center of an atom called?\nAssistant: The center of an atom is called the nucleus."},
    {"text": "User: What is the chemical symbol for iron?\nAssistant: The chemical symbol for iron is Fe."},
    {"text": "User: What part of the plant conducts photosynthesis?\nAssistant: The leaves conduct photosynthesis."},
    {"text": "User: Who was the first President of the United States?\nAssistant: George Washington was the first President of the United States."},
    {"text": "User: What do bees collect from flowers?\nAssistant: Bees collect nectar from flowers."}
]

dataset = Dataset.from_list(general_knowledge)

def format_prompt(example):
    user_part, assistant_part = example["text"].split("Assistant:")
    return {
        "text": f"<|im_start|>user\n{user_part.replace('User:', '').strip()}\n<|im_end|>\n<|im_start|>assistant\n{assistant_part.strip()}\n<|im_end|>"
    }

dataset = dataset.map(format_prompt)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [15]:
# Step 3C: Initial fine-tuning

training_args = TrainingArguments(
    output_dir="sft-initial-checkpoint",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    max_steps=100,
    logging_steps=5,
    save_strategy="no",
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=training_args,
)

trainer.train()


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 4 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,505,600/4,000,000,000 (0.11% trained)


Step,Training Loss
5,0.767300
10,0.511200
15,0.449500
20,0.417500
25,0.345100
30,0.377000
35,0.317000
40,0.296500
45,0.315100
50,0.294400


TrainOutput(global_step=100, training_loss=0.34545231580734254, metrics={'train_runtime': 44.9932, 'train_samples_per_second': 4.445, 'train_steps_per_second': 2.223, 'total_flos': 73138553929728.0, 'train_loss': 0.34545231580734254})

In [16]:
# Step 3D: Save initial checkpoint

model.save_pretrained("sft-initial-checkpoint")
tokenizer.save_pretrained("sft-initial-checkpoint")


('sft-initial-checkpoint/tokenizer_config.json',
 'sft-initial-checkpoint/special_tokens_map.json',
 'sft-initial-checkpoint/tokenizer.model',
 'sft-initial-checkpoint/added_tokens.json',
 'sft-initial-checkpoint/tokenizer.json')

In [17]:
# Step 4A: Load custom checkpoint

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="sft-initial-checkpoint",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
    device_map={"": 0},
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
# Step 4B: Domain support fine-tuning data

domain_data = [
    {"text": "User: How do I view my billing history?\nAssistant: You can view your billing history under the account settings."},
    {"text": "User: Is there an option to pause my subscription?\nAssistant: Yes, you can pause your subscription from your account dashboard."},
]

dataset = Dataset.from_list(domain_data)
dataset = dataset.map(format_prompt)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [19]:
# Step 4C: Continue fine-tuning

training_args = TrainingArguments(
    output_dir="sft-continued-checkpoint1",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    max_steps=100,
    logging_steps=5,
    save_strategy="no",
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=training_args,
)

trainer.train()

# Save checkpoint
model.save_pretrained("sft-continued-checkpoint1")
tokenizer.save_pretrained("sft-continued-checkpoint1")


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 100 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,505,600/4,000,000,000 (0.11% trained)


Step,Training Loss
5,0.243800
10,0.067500
15,0.019900
20,0.015500
25,0.014200
30,0.013500
35,0.013100
40,0.012900
45,0.012800
50,0.012700


('sft-continued-checkpoint1/tokenizer_config.json',
 'sft-continued-checkpoint1/special_tokens_map.json',
 'sft-continued-checkpoint1/tokenizer.model',
 'sft-continued-checkpoint1/added_tokens.json',
 'sft-continued-checkpoint1/tokenizer.json')

In [11]:
# Step 5A: Load previous checkpoint

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="sft-continued-checkpoint1",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
    device_map={"": 0},
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [12]:
# Step 5B: Customer service dialogues

customer_service = [
    {"text": "User: How can I delete my account?\nAssistant: You can delete your account by contacting customer support."},
    {"text": "User: Can I get a refund for a canceled order?\nAssistant: Refunds are processed automatically for eligible cancellations."},
]

dataset = Dataset.from_list(customer_service)
dataset = dataset.map(format_prompt)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [13]:
# Step 5C: Continue fine-tuning again

training_args = TrainingArguments(
    output_dir="sft-continued-checkpoint2",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    max_steps=100,
    logging_steps=5,
    save_strategy="no",
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=training_args,
)

trainer.train()

# Save final checkpoint
model.save_pretrained("sft-continued-checkpoint2")
tokenizer.save_pretrained("sft-continued-checkpoint2")


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 100 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,505,600/4,000,000,000 (0.11% trained)


Step,Training Loss
5,0.727200
10,0.289500
15,0.139500
20,0.057800
25,0.027200
30,0.019300
35,0.016400
40,0.015000
45,0.014200
50,0.013800


('sft-continued-checkpoint2/tokenizer_config.json',
 'sft-continued-checkpoint2/special_tokens_map.json',
 'sft-continued-checkpoint2/tokenizer.model',
 'sft-continued-checkpoint2/added_tokens.json',
 'sft-continued-checkpoint2/tokenizer.json')

In [20]:
# Step 6: Final testing after multiple fine-tunings

def generate_response_final(prompt, max_tokens=128):
    chat_prompt = f"<|im_start|>user\n{prompt}\n<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

# Inference
print("\nTest 1:")
print(generate_response_final("Can I pause my subscription temporarily?"))

print("\nTest 2:")
print(generate_response_final("How can I get a refund for a canceled order?"))



Test 1:
There is no cancellation fee for paused subscriptions.
<|im

Test 2:
There is a $20 cancellation fee for an invalid order.
